In [ ]:
from datasets import load_dataset, DatasetDict
from huggingface_hub import notebook_login
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from datasets import Audio
from transformers import WhisperForConditionalGeneration
import evaluate
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa
import csv
import matplotlib.pyplot as plt
import shutil
import os
from google.colab import drive
import shutil


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 29 18:15:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              50W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 129.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 67.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
tensorflow 2.17.1 requires tensorboard<2.18,>=2.17, but you have tensorboard 2.18.0 which is incompatible.


In [ ]:

notebook_login()

In [ ]:
common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "ar", split="train[:2%]", trust_remote_code=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "ar", split="test[:2%]", trust_remote_code=True)

print(common_voice)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 12910it [00:00, 129079.50it/s]
Reading metadata...: 35514it [00:00, 135757.37it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14959it [00:00, 139565.69it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 561
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 209
    })
})


In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 561
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 209
    })
})


In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Arabic", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Arabic", task="transcribe")

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/03e3da23e5fa2d7de56094c40cc7ea529b3b571b215978f053d1fd26557901a6/ar_train_0/common_voice_ar_24082672.mp3', 'array': array([-2.13162821e-14, -1.01252340e-13, -3.55271368e-14, ...,
        1.26173454e-08,  1.54317320e-07,  1.32340631e-07]), 'sampling_rate': 48000}, 'sentence': 'وما أدراك ما يوم الدين'}


In [ ]:
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/03e3da23e5fa2d7de56094c40cc7ea529b3b571b215978f053d1fd26557901a6/ar_train_0/common_voice_ar_24082672.mp3', 'array': array([ 8.73114914e-11, -4.36557457e-11,  1.67347025e-10, ...,
       -2.33121682e-08, -2.25205440e-07,  4.55183908e-08]), 'sampling_rate': 16000}, 'sentence': 'وما أدراك ما يوم الدين'}


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/561 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/209 [00:00<?, ? examples/s]

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
model.generation_config.language = "Arabic"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=150,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #tokenizer=None,  # Avoid deprecated argument
    processing_class=processor,  # Use the processor instead
)

In [ ]:
processor.tokenizer.pad_token = "<pad>"
processor.tokenizer.eos_token = "</s>"
processor.save_pretrained(training_args.output_dir)

[]

In [ ]:
model.save_pretrained(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


[]

In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
500,0.013000,0.968758,65.242165


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use

TrainOutput(global_step=500, training_loss=0.18835290837287902, metrics={'train_runtime': 1099.8281, 'train_samples_per_second': 7.274, 'train_steps_per_second': 0.455, 'total_flos': 2.2524090494976e+18, 'train_loss': 0.18835290837287902, 'epoch': 13.88888888888889})

In [ ]:
# Load the processor and model
processor = WhisperProcessor.from_pretrained("./whisper-small-hi")
model = WhisperForConditionalGeneration.from_pretrained("./whisper-small-hi")

# Prepare test input
test_audio_path = "/content/download.wav"

# Load the audio data using librosa
audio_data, sampling_rate = librosa.load(test_audio_path, sr=16000)  # Load and resample

# Process the audio data using the processor
audio_input = processor(audio_data, sampling_rate=sampling_rate, return_tensors="pt")  # Pass audio data

# Generate transcription
generated_ids = model.generate(audio_input.input_features)
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
print("Transcription:", transcription)

In [ ]:
losses, wers = [], []

for step, logs in enumerate(trainer.state.log_history):
    if "loss" in logs:
        losses.append((step, logs["loss"]))
    if "eval_wer" in logs:
        wers.append((step, logs["eval_wer"]))

# Save metrics to CSV
with open("metrics.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Step", "Loss", "WER"])
    for step, loss in losses:
        writer.writerow([step, loss, None])
    for step, wer in wers:
        writer.writerow([step, None, wer])

# Visualize separately
steps_loss, loss_vals = zip(*losses)
steps_wer, wer_vals = zip(*wers)

# Plot loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)  # Create a subplot for loss
plt.plot(steps_loss, loss_vals, label="Loss")
plt.xlabel("Step")
plt.ylabel("Loss Value")
plt.legend()
plt.title("Training Loss")

# Plot WER
plt.subplot(1, 2, 2)  # Create a subplot for WER
plt.plot(steps_wer, wer_vals, label="WER")
plt.xlabel("Step")
plt.ylabel("WER Value")
plt.legend()
plt.title("Evaluation WER")

plt.tight_layout()  # Adjust subplot spacing
plt.show()

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "hi",
    "model_name": "Whisper Small Hi - ZALOOM",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(**kwargs)

CommitInfo(commit_url='https://huggingface.co/mozaloom/whisper-small-hi/commit/7f4c2b55ad13ec87ccf977df7ae83068715f1d07', commit_message='End of training', commit_description='', oid='7f4c2b55ad13ec87ccf977df7ae83068715f1d07', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mozaloom/whisper-small-hi', endpoint='https://huggingface.co', repo_type='model', repo_id='mozaloom/whisper-small-hi'), pr_revision=None, pr_num=None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_output_dir = "/content/drive/MyDrive/SR-Project"

In [ ]:
# Save model and processor locally
trainer.save_model(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)

# Check if the directory exists, and remove it if it does
if os.path.exists(drive_output_dir):
    shutil.rmtree(drive_output_dir)

# Copy files to Google Drive
shutil.copytree(training_args.output_dir, drive_output_dir)
print(f"Model saved to Google Drive at {drive_output_dir}")

No files have been modified since last commit. Skipping to prevent empty commit.


Model saved to Google Drive at /content/drive/MyDrive/SR-Project


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Copy the model to a local directory
model_dir = "/content/drive/MyDrive/SR-Project"
local_model_dir = "./model"
shutil.copytree(model_dir, local_model_dir)

print(f"Model copied to {local_model_dir}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model copied to ./model


In [ ]:
# prompt: write an test cell

# Test the model with a sample audio file.
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Load the processor and model
processor = WhisperProcessor.from_pretrained("./model")
model = WhisperForConditionalGeneration.from_pretrained("./model")


def transcribe_audio(audio_path):
    # Load the audio data using librosa
    try:
        audio_data, sampling_rate = librosa.load(audio_path, sr=16000)
    except Exception as e:
        print(f"Error loading audio file: {e}")
        return None
    # Process the audio data using the processor
    audio_input = processor(audio_data, sampling_rate=sampling_rate, return_tensors="pt")
    # Generate transcription
    try:
      generated_ids = model.generate(audio_input.input_features)
      transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
      return transcription[0]
    except Exception as e:
      print(f"Error generating transcription: {e}")
      return None


# Example usage:
test_audio_path = "/content/ElevenLabs_2024-03-24T15_30_27_Rachel_pre_s50_sb75_se0_b_m2.mp3" # Replace with your test audio file
transcription = transcribe_audio(test_audio_path)

if transcription:
    print("Transcription:", transcription)

Transcription: بإمكانكم اتبعونا أول بأول على صفحتنا وتتواصلوا معنا عن طريق المسجات إذا بحاجة لأي سؤال


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa  # Import librosa for audio loading

# Load the processor and model
processor = WhisperProcessor.from_pretrained("./whisper-small-hi")
model = WhisperForConditionalGeneration.from_pretrained("./whisper-small-hi")

# Prepare test input
test_audio_path = "/content/ElevenLabs_2024-03-24T15_30_27_Rachel_pre_s50_sb75_se0_b_m2.mp3"

# Load the audio data using librosa
audio_data, sampling_rate = librosa.load(test_audio_path, sr=16000)  # Load and resample

# Process the audio data using the processor
audio_input = processor(audio_data, sampling_rate=sampling_rate, return_tensors="pt")  # Pass audio data

# Generate transcription
generated_ids = model.generate(audio_input.input_features)
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
print("Transcription:", transcription)

Transcription: ['بإمكانكم اتبعونا أول بأول على صفحتنا وتتواصلوا معنا عن طريق المسجات إذا بحاجة لأي سؤال']


In [ ]:
!pip install nemo

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 133.9 MB/s eta 0:00:00
  Created wheel for drf-flex-fields: filename=drf_flex_fields-1.0.2-py3-none-any.whl size=16641 sha256=41799472d31ab33d664df466079e4d61acc4997223ac9fbe951dbf3465fc76a3
  Stored in directory: /root/.cache/pip/wheels/15/9e/6c/95a7cd990d4561e61f3dd0b8408e33d0de3de70c2c1ebfb40b
Successfully built drf-flex-fields
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.2
    Uninstalling pytz-2024.2:
      Successfully uninstalled pytz-2024.2
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstalling py